In [1]:
import torch
import torchvision
import numpy as np

In [2]:
from cifar10models import *

In [4]:
model = SENet18()

In [5]:
model.load_state_dict(torch.load('../../../../share/cuvl/weights/cifar10/senet18_epoch_279_acc_94.59.pth'))

In [6]:
from attacks import ifgsm

In [7]:
import torchvision.datasets as dset

In [8]:
def _data_transforms_cifar10():
  CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
  CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

  train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
  ])

  valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    ])
  return train_transform, valid_transform

In [9]:
train_transform, valid_transform = _data_transforms_cifar10()

In [10]:
train_data = dset.CIFAR10(root='../data', train=True, download=True, transform=train_transform)

Files already downloaded and verified


In [11]:
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(0.5 * num_train))

In [12]:
train_tensors = []
train_label = []

In [13]:
for i in range(num_train):
    train_tensors.append(train_data[i][0])
    train_label.append(train_data[i][1])

In [14]:
train_tensors[5].size()

torch.Size([3, 32, 32])

In [15]:
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(0.5 * num_train))

train_queue = torch.utils.data.DataLoader(
  train_data, batch_size=64,
  sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:]),
  pin_memory=True, num_workers=2)

In [16]:
import torch.nn as nn

def ifgsm(model, X, y, niters=10, epsilon=0.01, visualize=False):
    X_pert = X.clone()
    X_pert.requires_grad = True
    
    for _ in range(niters):
        output_perturbed = model(X_pert)
        loss = nn.CrossEntropyLoss()(output_perturbed, y)
        loss.backward()
        pert = epsilon * X_pert.grad.detach().sign()

        # add perturbation
        X_pert = X_pert.detach() + pert
        X_pert.requires_grad = True
        
        # make sure we don't modify the original image beyond epsilon
        X_pert = (X_pert.detach() - X.clone()).clamp(-epsilon, epsilon) + X.clone()
        X_pert.requires_grad = True
        
        # adjust to be within [-1, 1]
        X_pert = X_pert.detach().clamp(-1, 1)
        X_pert.requires_grad = True
        
    return X_pert

In [17]:
from torch.autograd import Variable
perturbed_images = []
perturbed_labels = []
for step, (input, target) in enumerate(train_queue):
    input = Variable(input)
    target = Variable(target)
    
    input = ifgsm(model, input, target, niters=3)
    perturbed_images.append(input)
    perturbed_labels.append(target)

/share/apps/anaconda3/5.2.0/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [18]:
pert_imgs = torch.cat(perturbed_images)
pert_lbls = torch.cat(perturbed_labels)

In [19]:
pert_imgs.size()

torch.Size([50000, 3, 32, 32])

In [20]:
pert_lbls.size()

torch.Size([50000])

In [21]:
np.save('senet_imgs.npy', pert_imgs.detach().numpy())

In [22]:
np.save('senet_lbls.npy', pert_lbls.detach().numpy())